<a href="https://colab.research.google.com/github/sakethc4/FIRE-Research-Matrix-Multiplication/blob/main/Saketh_FIRE_MulPro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! git clone https://github.com/sakethc4/FIRE-Research-Matrix-Multiplication
! cd FIRE-Research-Matrix-Multiplication/

Cloning into 'FIRE-Research-Matrix-Multiplication'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 67 (delta 20), reused 27 (delta 3), pack-reused 0 (from 0)
Receiving objects: 100% (67/67), 502.35 KiB | 14.77 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [ ]:
!pip install pennylane==0.33.1 pennylane-qiskit==0.33.1 memory-profiler line-profiler

INFO: pip is looking at multiple versions of pennylane-lightning to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 48.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.6/717.6 kB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 69.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 73.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 91.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 80.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB

In [ ]:
from memory_profiler import profile  # For memory profiling
import pennylane as qml  # PennyLane for quantum machine learning
import pennylane.numpy as np  # PennyLane's NumPy wrapper for differentiable operations
import time  # For tracking execution time
from functools import wraps  # For creating decorators
from line_profiler import LineProfiler  # For line-by-line profiling
import csv  # For writing results to a CSV file

# Defining the quantum device with required qubits
# `wires_m` are the wires representing the multiplicand (m)
# `wires_solution` are the wires representing the solution for quantum Fourier transforms
wires_m = [0, 1]
wires_solution = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]

# Initialize a quantum device
dev = qml.device("default.qubit", wires=wires_m + wires_solution, shots=1)

n_wires = len(dev.wires)  # Calculate the total number of wires (qubits)

# Decorator to track the execution time for addition operations
def calculate_time_add(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()  # Record the start time
        result = func(*args, **kwargs)  # Call the actual function
        end = time.time()  # Record the end time
        execution_time = (end - start) * 1000  # Convert time to milliseconds
        global add_time
        add_time += execution_time  # Accumulate the time spent in addition operations
        return result
    return wrapper

# Decorator to track the execution time for multiplication operations
def calculate_time_mul(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        execution_time = (end - start) * 1000  # Convert time to milliseconds
        global mul_time
        mul_time += execution_time  # Accumulate the time spent in multiplication operations
        return result
    return wrapper

# Decorator to track the total execution time of matrix multiplication
def calculate_time(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        execution_time = (end - start) * 1000  # Convert time to milliseconds
        global total_time
        total_time = execution_time  # Store the total execution time
        return result
    return wrapper

# Function to perform a quantum Fourier-based addition on a quantum circuit
def add_k_fourier(k, wires):
    for j in range(len(wires)):
        qml.RZ(k * np.pi / (2**j), wires=wires[j])  # Apply rotation to each qubit

# Function to perform controlled multiplication using quantum Fourier transforms
def multiplication(k, wires_m, wires_solution):
    for i in range(len(wires_m) - 1, -1, -1):
        qml.ctrl(add_k_fourier, control=wires_m[i])(k, wires_solution[len(wires_m) - 1 - i:])

# Quantum node for multiplication operation
@calculate_time_mul
@qml.qnode(dev)
def mul(m, k):
    qml.BasisEmbedding(m, wires=wires_m)  # Embedding `m` into the quantum circuit
    qml.QFT(wires=wires_solution)  # Apply Quantum Fourier Transform (QFT)
    multiplication(k, wires_m, wires_solution)  # Perform controlled multiplication
    qml.adjoint(qml.QFT)(wires=wires_solution)  # Apply the inverse QFT
    return qml.sample(wires=wires_solution)  # Return the measurement samples

# Quantum node for summation operation using QFT
@calculate_time_add
@qml.qnode(dev)
def sum3(m, k):
    qml.BasisEmbedding(m, wires=range(len(wires_solution)))  # Embedding `m` into quantum circuit
    qml.QFT(wires=range(len(wires_solution)))  # Step 1: Apply QFT
    add_k_fourier(k, range(len(wires_solution)))  # Step 2: Apply addition in Fourier space
    qml.adjoint(qml.QFT)(wires=range(len(wires_solution)))  # Step 3: Inverse QFT
    return qml.sample(wires=range(len(wires_solution)))  # Return the measurement samples

# Initialize matrices with specific values
def init_values(k, val):
    global matrix_a, matrix_b, matrix_c, n, m
    matrix_a = np.full((k, k), val)  # Create matrix A filled with value `val`
    matrix_b = np.full((k, k), val)  # Create matrix B filled with value `val`
    n = matrix_a.shape[0]  # Number of rows in matrix A
    m = matrix_b.shape[1]  # Number of columns in matrix B
    matrix_c = np.zeros((n, m))  # Initialize matrix C for the result

# Function to compute dot product using quantum multiplication and summation
def dot(a, b):
    listx = []
    x = 0
    for i in range(a.shape[0]):
        mulnum = mul(int(a[i]), int(b[i]))  # Perform quantum multiplication
        mulnum = int(''.join(map(str, list(mulnum))), 2)  # Convert binary output to decimal
        listx.append(mulnum)
    i = 0
    while (listx.__len__() > 1):  # While more than one element in the list
        i = 0
        sumnum = sum3(listx[i], listx[i + 1])  # Perform quantum summation
        sumnum = int(''.join(map(str, list(sumnum))), 2)  # Convert binary output to decimal
        x = x + sumnum  # Accumulate the sum
        i = i + 1
        if listx.__len__() >= 2:
            listx = listx[2:]  # Remove the two summed values from the list
            listx.insert(0, sumnum)  # Insert the sum result at the start
    return listx[0]  # Return the final sum result

# Main matrix multiplication function
@calculate_time
def matrix_multiplication_mulpro():
    for i in range(n):
        for j in range(m):
            matrix_c[i, j] = dot(matrix_a[i], matrix_b[:, j])  # Compute the dot product
            print(matrix_c)  # Print the resulting matrix after each operation

if __name__ == '__main__':
    # Open CSV file to store the results
    csv_file = open('multication_mulpro.csv', 'a', newline='')

    # Loop through different matrix dimensions (2x2, 4x4, etc.)
    for matrix_dimension in [2, 4, 8, 16]:
        init_values(matrix_dimension, 2)  # Initialize matrices with the specified dimension and value
        add_time, mul_time, total_time = 0, 0, 0  # Reset times for each matrix size
        print("--------- MATRIX OF SIZE ", matrix_dimension, "---------")
        matrix_multiplication_mulpro()  # Perform the matrix multiplication

        # Write execution times to the CSV file
        writer = csv.writer(csv_file)
        data = [['size', 'value', 'add_time', 'mul_time', 'total_time'],
                [matrix_dimension, 2, add_time, mul_time, total_time]]
        writer.writerows(data)

    csv_file.close()  # Close the CSV file

--------- MATRIX OF SIZE  2 ---------
[[8. 0.]
 [0. 0.]]
[[8. 8.]
 [0. 0.]]
[[8. 8.]
 [8. 0.]]
[[8. 8.]
 [8. 8.]]
--------- MATRIX OF SIZE  4 ---------
[[16.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16. 16.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16. 16. 16.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16. 16. 16.]
 [16.  0.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16. 16. 16.]
 [16. 16.  0.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16. 16. 16.]
 [16. 16. 16.  0.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16. 16. 16.]
 [16. 16. 16. 16.]
 [ 0.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16. 16. 16.]
 [16. 16. 16. 16.]
 [16.  0.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16. 16. 16.]
 [16. 16. 16. 16.]
 [16. 16.  0.  0.]
 [ 0.  0.  0.  0.]]
[[16. 16. 16. 16.]
 [16. 16. 16. 16.]
 [16. 16. 16.  0.]
 [ 0.  0.  0.  0.]]
[